# **Homework 2**
**Name:** Amirhossein Jamali

**GitHub repository:** https://github.com/jamir91/NLP

In [1]:
class DataPreprocessor:
    data = None

    def __init__(self):
        pass

    def read_data(self):
        pass

    def plot_distribution(self):
        pass

    def clean_text(self):
        pass

    def count_words(self):
        pass

    def mao_word_index(self):
        pass

    def tokenize(self):
        pass

In [3]:
import pandas as pd

train_df = pd.read_csv('./data/train.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117192 entries, 0 to 117191
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   link       117192 non-null  object
 1   title      117161 non-null  object
 2   code_news  117161 non-null  object
 3   category   117161 non-null  object
 4   date       117161 non-null  object
 5   text       97030 non-null   object
dtypes: object(6)
memory usage: 5.4+ MB
